In [1]:
import pandas as pd
import requests
import json

from math import radians, sin, cos, sqrt, atan2

data_store = {
    'muzei.csv': 'https://classif.gov.spb.ru/irsi/7842489089-muzei/structure_version/569/?page=1&per_page=10',
    'teatry.csv': 'https://classif.gov.spb.ru/irsi/7842489089-teatry/structure_version/649/',
    'restorany.csv': 'https://classif.gov.spb.ru/irsi/7842489089-restorany/structure_version/570/',
    'informaciya-o-nacionalno-kulturnyh-obuedineniyah-nacionalno-kulturnyh-avtonomiyah-i-kazachih-obshestvah-sankt-peterburga.csv': 'https://classif.gov.spb.ru/irsi/7842510855-informaciya-o-nacionalno-kulturnyh-obuedineniyah-nacionalno-kulturnyh-avtonomiyah-i-kazachih-obshestvah-sankt-peterburga/structure_version/310/',
    'Katki-i-lyzhnye-trassy-Sankt-Peterburga.csv': 'https://classif.gov.spb.ru/irsi/7814348015-Katki-i-lyzhnye-trassy-SPbtwo/structure_version/618/',
    'Vnutridvorovye-sportivnye-ploshadki-Sankt-Peterburga.csv': 'https://classif.gov.spb.ru/irsi/7814348015-Vnutridvorovye-sportivnye-ploshadki-SPb/structure_version/591/',
    'address_radar.csv': 'https://classif.gov.spb.ru/irsi/7825457753-address_radar/structure_version/568/',
    'vystavochnye-zaly.csv': 'https://classif.gov.spb.ru/irsi/7842489089-vystavochnye-zaly/structure_version/151/',
    'informaciya-o-gosudarstvennyh-kazennyh-uchrezhdeniyah-podvedomstvennyh-arhivnomu-komitetu-sankt-peterburga.csv' : 'https://classif.gov.spb.ru/irsi/7812017177-informaciya-o-gosudarstvennyh-kazennyh-uchrezhdeniyah-podvedomstvennyh-arhivnomu-komitetu-sankt-peterburga/structure_version/401/',
    'Spisok-specializirovannyh-sadovo-parkovyh-predpriyatij.csv' : 'https://classif.gov.spb.ru/irsi/7840424142-Spisok-specializirovannyh-sadovo-parkovyh-predpriyatij/structure_version/148/'
}

def calculate_distance(coord1, coord2):
    lat1, lon1 = map(float, coord1.split(','))
    lat2, lon2 = coord2[1], coord2[0]

    # Радиус Земли в метрах
    radius = 6371000

    # Преобразование координат в радианы
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Разница между долготами и широтами
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Формула Винсенти для расчета расстояния на сфере
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = radius * c

    return distance

def reverse_geocode_coordinates(coordinates):
    url = 'https://geocode-maps.yandex.ru/1.x/'
    lat1, lon1 = map(float, coordinates.split(','))
    params = {'geocode': f'{lon1},{lat1}', 'format': 'json','apikey': 'd759fc98-e49c-4757-a527-df4c301a81a5'}
    response = requests.get(url, params=params)
    data = json.loads(response.text)
    address = None
    if 'response' in data and 'GeoObjectCollection' in data['response']:
        collection = data['response']['GeoObjectCollection']
        if 'featureMember' in collection and len(collection['featureMember']) > 0:
            feature_member = collection['featureMember'][0]
            if 'GeoObject' in feature_member and 'metaDataProperty' in feature_member['GeoObject']:
                meta_data = feature_member['GeoObject']['metaDataProperty']
                if 'GeocoderMetaData' in meta_data and 'Address' in meta_data['GeocoderMetaData']:
                    address = meta_data['GeocoderMetaData']['Address']['formatted']
    return address

def geocode_address(address):
    url = 'https://geocode-maps.yandex.ru/1.x/'
    params = {'geocode': address, 'format': 'json','apikey': 'd759fc98-e49c-4757-a527-df4c301a81a5'}
    response = requests.get(url, params=params)
    data = json.loads(response.text)
    coordinates = None
    faddress = None
    if 'response' in data and 'GeoObjectCollection' in data['response']:
        collection = data['response']['GeoObjectCollection']
        if 'featureMember' in collection and len(collection['featureMember']) > 0:
            feature_member = collection['featureMember'][0]
            if 'GeoObject' in feature_member and 'Point' in feature_member['GeoObject']:
                point = feature_member['GeoObject']['Point']
                coordinates = tuple(map(float, point['pos'].split()))
            if 'GeoObject' in feature_member and 'metaDataProperty' in feature_member['GeoObject']:
                meta_data = feature_member['GeoObject']['metaDataProperty']
                if 'GeocoderMetaData' in meta_data and 'Address' in meta_data['GeocoderMetaData']:
                    faddress = meta_data['GeocoderMetaData']['Address']['formatted']
    structe = [coordinates,faddress]    
    return structe
    

def analyze_data(file_name, address_column, coordinates_column):
    data = pd.read_csv(file_name)
    total_records = len(data)
    non_empty_addresses = data[address_column].count()
    non_empty_coordinates = data[coordinates_column].count()

    matched_address_to_coordinates = 0
    unprocessable_addresses = 0
    address_package = []

    for index, row in data.iterrows():
        address = row[address_column]
        coordinates = row[coordinates_column]

        # Геокодирование адреса и проверка соответствия координат
        structe = geocode_address(address)
        geocoded_coordinates = structe[0]
        address_package.insert(index, structe[1])
        if geocoded_coordinates is not None:
            distance = calculate_distance(coordinates, geocoded_coordinates)
            if distance <= 50:
                matched_address_to_coordinates += 1
        else:
            unprocessable_addresses += 1

    matched_coordinates_to_address_new = 0
    for index, row in data.iterrows():
        coordinates = row[coordinates_column]
        address_from_coordinates = reverse_geocode_coordinates(coordinates)
        if address_from_coordinates is not None: 
            if address_from_coordinates == address_package[index]:
                matched_coordinates_to_address_new += 1
        else:
            unprocessable_addresses += 1
    fname =  (file_name.split('/'))[1]

    output_data = {
        'название набора данных': file_name,
        'ссылка на набор данных': f'{data_store[fname]}',
        'количество записей в наборе': total_records,
        'название столбца с адресом': address_column,
        'название столбца с координатами': coordinates_column,
        'количество не пустых адресов': non_empty_addresses,
        'количество не пустых координат': non_empty_coordinates,
        'количество координат, совпавших с результатом геокодирования адреса': matched_address_to_coordinates,
        'количество адресов, совпавших с результатом геокодирования координат': matched_coordinates_to_address_new,
        'количество адресов, которые не удалось геокодировать': unprocessable_addresses
    }

    output_table = pd.DataFrame([output_data])
    return output_table

file_name = 'Данные/Spisok-specializirovannyh-sadovo-parkovyh-predpriyatij.csv'
address_column = 'Адрес'
coordinates_column = 'Координаты'

output_table = analyze_data(file_name, address_column, coordinates_column)